## Метод потенциалов для решения матричной транспортной задачи
## Выполнил: Яковлев Артур, 853501
## Проверила: Костюкова О.И.

### Вариант 28

Импорт необходимых библиотек

In [1]:
import numpy as np
import scipy.linalg as sla
from copy import deepcopy

Исходные данные для 28 варианта задания

In [2]:
dataset = {
    'a': np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
    'b': np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
    'c': np.array([
        [  5.,  3.,  40., -2.,  -3.,  -6., -2.,  2.,  3., 11., -19., -1.],
        [  4.,  3.,   4.,  6.,  -3.,   6., -7.,  8., -9., 10.,  -7.,  2.],
        [ -5.,  9.,  20., -2.,  -3.,   6.,  7.,  8.,  0.,  0.,   1., -2.],
        [ -2.,  3.,   0.,  4.,  -3.,   0., -1.,  4., -9.,  2.,   1.,  2.],
        [ -4., -2.,  14.,  2.,   3.,   6.,  0.,  1., -4., 10.,  -1.,  2.],
        [  2., -3.,   4.,  0.,  -3.,   4.,  7.,  8., -9.,  5.,   1., -8.],
        [  1.,  3.,   0.,  2.,   3.,   2.,  4.,  1.,  0., -1.,  -3., -2.],
        [ -5., -3.,   3., -2.,  -3., -16.,  7.,  2., -9.,  1.,   1.,  2.],
        [ -1.,  3.,   4., -1.,  -3.,   6.,  0.,  0., -9.,  6.,   1.,  6.],
        [-12., -2.,   0., -2.,  -3.,   6.,  7.,  8.,  9.,  1.,  10.,  2.],
        [  3.,  2.,   1., -9., -13., -16.,  4.,  1.,  3., 11.,  15.,  2.],
        [ -3.,  5.,  12.,  4., -14., -15., 21., 11., -1., 22.,  12.,  2.],
    ]),
}
eps = 1e-9

Реализация метода северо-западного угла для поиска базисного плана задачи

In [3]:
def north_west(dataset):
    a = deepcopy(dataset['a'])
    b = deepcopy(dataset['b'])
    c = deepcopy(dataset['c'])
    i = 0
    j = 0
    plan = {}
    x = np.zeros(c.shape)
    for _ in range(a.shape[0] + b.shape[0] - 1):
        if i + 1 == a.shape[0]:
            x[i, j] = b[j]
        elif j + 1 == b.shape[0]:
            x[i, j] = a[i]
        else:
            x[i, j] = min(a[i], b[j])
        plan[(i, j)] = x[i, j]
        if x[i, j] == a[i]:
            b[j] -= x[i, j]
            i += 1
        else:
            a[i] -= x[i, j]
            j += 1
    return plan

Реализация метода минимального элемента для поиска базисного плана задачи

In [4]:
def min_element(dataset):
    a = deepcopy(dataset['a'])
    b = deepcopy(dataset['b'])
    c = np.array(deepcopy(dataset['c']))
    plan = {}
    for _ in range(a.shape[0] + b.shape[0] - 1):
        min_value = np.argmin(c)
        j, i = min_value % b.shape[0], min_value // b.shape[0]
        if c[i, j] == np.inf:
            break
        x = min(a[i], b[j])
        if x == b[j]:
            c[:, j] = np.inf
            a[i] -= x
        else:
            c[i, :] = np.inf
            b[j] -= x
        
        plan[(i, j)] = x
        
    
    while len(plan) != a.shape[0] + b.shape[0] - 1:
        for i in range(a.shape[0]):
            for j in range(b.shape[0]):
                if (i, j) not in plan and get_cycle(c, plan, i, j) is None:
                    plan[(i, j)] = 0.
                    break
    
    return plan

Один из возможных начальных планов для нашей задачи:

In [12]:
min_element(dataset)

{(0, 10): 1.0,
 (7, 5): 1.0,
 (11, 4): 1.0,
 (9, 0): 1.0,
 (1, 8): 1.0,
 (10, 3): 1.0,
 (5, 11): 1.0,
 (1, 6): 0.0,
 (5, 1): 0.0,
 (7, 1): 0.0,
 (0, 6): 0.0,
 (4, 1): 1.0,
 (3, 6): 1.0,
 (6, 9): 1.0,
 (3, 2): 0.0,
 (6, 2): 0.0,
 (8, 7): 1.0,
 (9, 2): 0.0,
 (10, 2): 0.0,
 (8, 2): 0.0,
 (11, 2): 0.0,
 (4, 2): 0.0,
 (2, 2): 1.0}

Далее реализуем ряд функций, необходимых для построения цикла, что понадобиться при использовании метода потенциалов.

In [5]:
def get_cycle_vertexes(matrix):
    while True:
        any_replaced = False
        for i in range(matrix.shape[0]):
            if np.sum(matrix[i] == 1.) == 1:
                matrix[i] = np.zeros(matrix.shape[1])
                any_replaced = True
        
        for i in range(matrix.shape[1]):
            if np.sum(matrix[:, i]) == 1:
                matrix[:, i] = np.zeros(matrix.shape[0])
                any_replaced = True
                
        if not any_replaced:
            break
        
    result = []
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if matrix[i, j] == 1:
                result.append((i, j))
    
    return result
        

def get_cycle(c, init_plan, i0, j0):
    plan = deepcopy(init_plan)
    c = np.zeros(c.shape)
    for i, j in init_plan.keys():
        c[i, j] = 1.
    
    c[i0, j0] = 1.
    vertexes = get_cycle_vertexes(c)
    if (i0, j0) not in vertexes:
        return None
    result = [(i0, j0)]
    vertexes.remove((i0, j0))
    while vertexes:
        for i, j in vertexes:
            if i == result[-1][0]:
                result.append((i, j))
                break
        vertexes.remove(result[-1])
        
        if not vertexes:
            break
        for i, j in vertexes:
            if j == result[-1][1]:
                result.append((i, j))
                break
        vertexes.remove(result[-1])
    
    return result

Теперь реализуем сам метод потенциалов. Функция кроме исходных данных принимает в качестве аргументов максимальное число итераций, а также количество итераций с подробным описанием. В данном примере будут выведены три первые итерации.

In [6]:
def solve(dataset, init_plan, max_iters=100, verbose=True, verbose_iter=3):
    a = deepcopy(dataset['a'])
    b = deepcopy(dataset['b'])
    c = deepcopy(dataset['c'])
    
    if verbose:
        print('\nInitial data\n')
        print(f'a = {a}')
        print(f'b = {b}')
        print(f'C = {c}')
        print(f'Initial plan: {init_plan}')
    
    for it in range(max_iters):
        
        if it >= verbose_iter:
            verbose = False
        
        if verbose:
            print('\n======================================\n')
            print(f'Iteration {it + 1}')
        
        # Step 1
        u, v = np.array([np.inf] * a.shape[0]), np.array([np.inf] * b.shape[0])
        u[0] = 0.
        while np.any(u == np.inf) or np.any(v == np.inf):
            for i in range(a.shape[0]):
                for j in range(b.shape[0]):
                    if (i, j) in init_plan:
                        if u[i] != np.inf and v[j] == np.inf:
                            v[j] = c[i, j] - u[i]
                        elif u[i] == np.inf and v[j] != np.inf:
                            u[i] = c[i, j] - v[j]
        
        current_cost = 0.
        for i, j in init_plan.keys():
            current_cost += c[i, j] * init_plan[(i, j)]
        
        if verbose:
            print('\nStep 1\n')
            print(f'u = {u}')
            print(f'v = {v}')
            print(f'current cost: {current_cost}')
        
        # Step 2
        delta_j = -c + u.reshape(1, -1)[0, :][:, np.newaxis] + v.reshape(-1, 1)[:, 0]
        for i, j in init_plan.keys():
            delta_j[i, j] = -np.inf
        
        if verbose:
            print('\nStep 2\n')
            print(f'delta j = {delta_j}')
        
        # Step 3
        if np.all(delta_j < eps):
            print(f'\nBest plan: {init_plan}\n')
            print(f'Iterations required: {it}')
            print(f'Cost: {current_cost}')
            return
        
        # Step 4
        i0, j0 = np.argmax(delta_j) // b.shape[0], np.argmax(delta_j) % b.shape[0]
        
        if verbose:
            print('\nStep 4\n')
            print(f'i0 = {i0}')
            print(f'j0 = {j0}')
        
        # Step 5
        cycle = get_cycle(c, init_plan, i0, j0)
        theta = np.inf
        i_ast, j_ast = -1, -1
        for i in range(1, len(cycle), 2):
            if theta > init_plan[cycle[i]]:
                i_ast = cycle[i][0]
                j_ast = cycle[i][1]
                theta = init_plan[(i_ast, j_ast)]
        
        if verbose:
            print('\nStep 5\n')
            print('Cycle:')
            print(cycle)
            print(f'theta = {theta}')
            print(f'i*, j* = {i_ast}, {j_ast}')
        
        # Step 6
        init_plan[(i0, j0)] = 0.
        for i in range(len(cycle)):
            if i % 2 == 0:
                init_plan[cycle[i]] += theta
            else:
                init_plan[cycle[i]] -= theta
        
        init_plan.pop((i_ast, j_ast))
        if verbose:
            print(f'Updated plan: {init_plan}')
    
    print(f'\nBest found plan: {init_plan}')
        

Решим задачу при поиске базисного плана методом северо-западного угла

In [14]:
solve(dataset, north_west(dataset), verbose=True)


Initial data

a = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
b = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
C = [[  5.   3.  40.  -2.  -3.  -6.  -2.   2.   3.  11. -19.  -1.]
 [  4.   3.   4.   6.  -3.   6.  -7.   8.  -9.  10.  -7.   2.]
 [ -5.   9.  20.  -2.  -3.   6.   7.   8.   0.   0.   1.  -2.]
 [ -2.   3.   0.   4.  -3.   0.  -1.   4.  -9.   2.   1.   2.]
 [ -4.  -2.  14.   2.   3.   6.   0.   1.  -4.  10.  -1.   2.]
 [  2.  -3.   4.   0.  -3.   4.   7.   8.  -9.   5.   1.  -8.]
 [  1.   3.   0.   2.   3.   2.   4.   1.   0.  -1.  -3.  -2.]
 [ -5.  -3.   3.  -2.  -3. -16.   7.   2.  -9.   1.   1.   2.]
 [ -1.   3.   4.  -1.  -3.   6.   0.   0.  -9.   6.   1.   6.]
 [-12.  -2.   0.  -2.  -3.   6.   7.   8.   9.   1.  10.   2.]
 [  3.   2.   1.  -9. -13. -16.   4.   1.   3.  11.  15.   2.]
 [ -3.   5.  12.   4. -14. -15.  21.  11.  -1.  22.  12.   2.]]
Initial plan: {(0, 0): 1.0, (1, 0): 0.0, (1, 1): 1.0, (2, 1): 0.0, (2, 2): 1.0, (3, 2): 0.0, (3, 3): 1.0, (4, 3): 0.0, (4, 4): 1.0, (5, 4): 

Как мы видим, полученная стоимость равна -96, и на решение потребовалась 33 итерации

Теперь решим задачу с базисным планом, построенным методом минимального элемента

In [7]:
solve(dataset, min_element(dataset), verbose=False)


Best plan: {(0, 10): 1.0, (7, 5): 1.0, (11, 4): 1.0, (9, 0): 1.0, (1, 8): 0.0, (10, 3): 1.0, (5, 11): 1.0, (1, 6): 1.0, (4, 1): 1.0, (6, 9): 0.0, (3, 2): 0.0, (6, 2): 1.0, (8, 7): 1.0, (9, 2): 0.0, (2, 9): 1.0, (5, 8): 0.0, (7, 8): 0.0, (11, 5): 0.0, (8, 8): 0.0, (10, 5): 0.0, (3, 8): 1.0, (4, 7): 0.0, (0, 7): 0.0}

Iterations required: 10
Cost: -96.0


В результате мы получили точно такую же стоимость, при этом понадобилось всего 10 итераций. Таким образом метод минимального элемента требует более значительных затрат при вычислении, однако обеспечивает гораздо более быструю сходимость